In [ ]:
#general instructions

#2 or more patient encounters are required as the authors remove the last encounter while making predictions
# patient mortality is the outcome label and aquired from mimic. 
#2825 out of 7537 patients died (37.9%)
#Data set was divided into  train (5275/7537), validation (753/7537) and test (1509/7537)
# Once the optimal parameters were selected, the model was retrained by combining train and validation (6028/7537)
#
#
#********************** stars mean the data is not correct but the implemention functions.
#

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import time
import sys
import pickle
import math
import argparse
from torch.autograd import Variable
from tqdm import tqdm
from __future__ import division
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split

In [ ]:
# set seed - only really necessary to compare results with each other, but can remove from the submission
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# define data path
DATA_PATH = "mimic-iii-clinical-database-1.4"


# define data - separated to show which modules load (some take much longer than others)
#The commented out sections take a long time to load, and weren't immidiately useful at this stage
#Can uncommment when relevant

#TODO
# - reorganize into a definition() after  

In [ ]:
PatientsInfo = pd.read_csv(DATA_PATH + '/PATIENTS.csv')

In [ ]:
Diagnosis = pd.read_csv(DATA_PATH + '/DIAGNOSES_ICD.csv')

In [ ]:
Procedures = pd.read_csv(DATA_PATH + '/D_ICD_PROCEDURES.csv')

In [ ]:
Prescriptions = pd.read_csv(DATA_PATH + '/PRESCRIPTIONS.csv', low_memory = False)

In [ ]:
LabEvents = pd.read_csv(DATA_PATH + '/LABEVENTS.csv')

In [ ]:
LabLabels = pd.read_csv(DATA_PATH + '/D_LABITEMS.csv')

In [ ]:
Admissions = pd.read_csv(DATA_PATH + '/ADMISSIONS.csv')

In [ ]:
ProcedureLabels = pd.read_csv(DATA_PATH + '/D_ITEMS.csv')

In [ ]:
IcuStays = pd.read_csv(DATA_PATH + '/ICUSTAYS.csv')

In [ ]:
#ChartEvents = pd.read_csv(DATA_PATH + '/CHARTEVENTS.csv')

In [ ]:
# patient inclusion - Only ICU patients with 2 or more visits are included. Patient IDs gathered
#May be incorrect as the paper specifies only 7,537 patients in their list.
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=sys.maxsize)

#********************************************
#Generate an ignore list for later ease of use.
#Manually verified that all the included patients have 2 or more subject ids. Refine by HADM_ID? ICUSTAY_ID?

def Patient_Inclusion(IcuStays):
    # Note that an encounter represents a single visit or 
    # admission of the patient to the hospital Intensive Care Unit (ICU)
    PatientVisits = IcuStays[['SUBJECT_ID', 'ICUSTAY_ID', 'HADM_ID']]
    print("Original data")
    print("ICU visits shape: ", PatientVisits.shape)
    print("Number of patients: ", len(PatientVisits.SUBJECT_ID.unique()))
    print()
    
    #HADM_ID_LOOKUP = IcuStays[['ICUSTAY_ID', 'HADM_ID']]
    #print("ICU visits shape: ", HADM_ID_LOOKUP.shape)
    #print("Number of icu stays: ", len(PatientVisits.ICUSTAY_ID.unique()))
    
    PatientVisits['VISIT_COUNT'] = PatientVisits.sort_values(['SUBJECT_ID', 'ICUSTAY_ID']).groupby(['SUBJECT_ID'])['ICUSTAY_ID'].transform('count')
    print("After counting visits per patient")
    print("ICU visits shape: ", PatientVisits.shape)
    print("Number of patients: ", len(PatientVisits.SUBJECT_ID.unique()))
    print()
    
    PatientVisits = PatientVisits.query('VISIT_COUNT > 1')
    print("After filtering for VISIT_COUNT > 1")
    print("ICU visits shape: ", PatientVisits.shape)
    print("Number of patients: ", len(PatientVisits.SUBJECT_ID.unique()))
    print()
    
    PatientVisits = PatientVisits[['SUBJECT_ID', 'ICUSTAY_ID', 'HADM_ID']]
    PatientSeries = PatientVisits.groupby("SUBJECT_ID")["ICUSTAY_ID"].apply(list)
    #This variable contains Subject_ID, Icustay_id and Hadm_id
    HadmId = PatientVisits
    
    Patients = PatientSeries.index.tolist()
    Visits = PatientSeries.tolist()

    return Patients, Visits, HadmId

In [ ]:
#To be added to main at the end
Patients, Visits, HadmId = Patient_Inclusion(IcuStays)

In [ ]:
#Statistics
maxcount = len(Visits[0])
mincount = len(Visits[0])
for i in Visits:
    if len(i) > maxcount:
        maxcount = len(i)
    if len(i) < mincount:
        mincount = mincount

print("Number of Patients: ", len(Patients))
print("Len of Visits: ", len(Visits))        
print("Max visits for a patient: ", maxcount)
print("Min visits for a patient: ", mincount)

In [ ]:
# data statistics - Ethnicity and visit statistics

def Ethnicity_Statistics(Admissions):
    white = 0
    black = 0
    hispanic = 0
    asian = 0
    other = 0
    
    #separate out ethnicity information
    EthStatistics = Admissions[['SUBJECT_ID', 'ETHNICITY']]
    IcuPatientEth = EthStatistics[EthStatistics.SUBJECT_ID.isin(Patients)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)

    #Not yet working.... General idea.
    for i in IcuPatientEth.ETHNICITY:
        if "WHITE" in i:
            white += 1
        if "BLACK" in i:
            black += 1
        if "HISPANIC" in i:
            hispanic += 1
        if "ASIAN"  in i:
            asian += 1
    
        #I tried combining it, but the count was always more wrong...
        if "OTHER" in i:
            other += 1
        if "DECLINED" in i:
            other += 1
        if "NATIVE" in i:
            other += 1
        if "MIDDLE EASTERN" in i:
            other += 1
        if "MULTI" in i:
            other += 1
        if "UNABLE" in i:
            other += 1
        if "UNKNOWN" in i:
            other += 1
        

    print(np.round(white / len(IcuPatientEth), 3), "percent white")
    print(np.round(black / len(IcuPatientEth), 3), "percent black")
    print(np.round(hispanic / len(IcuPatientEth), 3), "percent hispanic")
    print(np.round(asian / len(IcuPatientEth), 3), "percent asian")
    print(np.round(other / len(IcuPatientEth), 3), "percent other")

    print((np.round((white + black + hispanic + asian + other) / len(IcuPatientEth), 2)), "percent total")
    
    #*****************************************************
    #6 too many values. Need to sort that out.... maybe? it doesnt actually really affect anything...
    print(len(IcuPatientEth))
    print(white + black + asian + hispanic + other)
    return white, black, hispanic, asian, other

In [ ]:
#To be added to main at the end
white, black, hispanic, asian, other = Ethnicity_Statistics(Admissions)

In [ ]:
#Age statistics - Date of birth DOB is in patients, and admissions time is in admissions.
# need to correlate both to find patients true ages
#Sorting to ensure that each patient id is matched in case of misordering
def Age_statistics(PatientsInfo):

    #separate out date of birth information and sort
    AgeStatistics = PatientsInfo[['SUBJECT_ID', 'DOB']]
    IcuPatientDob = AgeStatistics[AgeStatistics.SUBJECT_ID.isin(Patients)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)
    IcuPatientDob = IcuPatientDob.sort_values(by=['SUBJECT_ID']).reset_index(drop = True)
    
    #separate out admission date and sort
    AdmStatistics = Admissions[['SUBJECT_ID', 'ADMITTIME']]
    IcuPatientAdm = AdmStatistics[AdmStatistics.SUBJECT_ID.isin(Patients)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)
    IcuPatientAdm = IcuPatientAdm.sort_values(by=['SUBJECT_ID']).reset_index(drop = True)
    
    #minus dates to find ages
    #********************************************************
    # not done yet. need to convert at least part of the date to an int for each dataframe
    #Year and month should be sufficient, but year month date would increase accuracy. remove time.
    
    
Age_statistics(PatientsInfo)

In [ ]:
#Sex statistics
def Gender_Statistics(PatientsInfo):
    male = 0
    female = 0
    
    SexStatistics = PatientsInfo[['SUBJECT_ID', 'GENDER']]
    IcuPatientSex = SexStatistics[SexStatistics.SUBJECT_ID.isin(Patients)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)
    
    for i in IcuPatientSex.GENDER:
        if i == "M":
            male += 1
        if i == "F":
            female += 1
            
    print("Male patients: ", male)
    print("Female patients: ", female)
    
    print(np.round(male / len(IcuPatientSex), 2), " percent male")
    print(np.round(female / len(IcuPatientSex), 2), "percent female")
    
    return male, female

In [ ]:
male, female = Gender_Statistics(PatientsInfo)

In [ ]:
#ICD Code statistics
def Icd_Codes(Diagnosis):
    
    #Isolate just the ICD codes and sequence of ICD codes
    IcdStatistics = Diagnosis[['SUBJECT_ID', 'HADM_ID', 'ICD9_CODE', 'SEQ_NUM']]
    IcdStatistics.SEQ_NUM = IcdStatistics.SEQ_NUM.dropna()
    IcuPatientIcd = IcdStatistics[IcdStatistics.SUBJECT_ID.isin(Patients)].reset_index(drop = True)
    
    #To calculate the total unique ICD codes present
    TotalIcdCode = IcuPatientIcd.filter(["ICD9_CODE"]).drop_duplicates()
    
    #For ICD code groupings by patient id
    PatientIcdCodes = IcuPatientIcd.groupby(["SUBJECT_ID", "HADM_ID"])["ICD9_CODE"].apply(list)
    
    #For overall ICD Statistics
    EncounterTotal = IcuPatientIcd.drop_duplicates(['SUBJECT_ID', 'SEQ_NUM']).groupby('SUBJECT_ID')['SEQ_NUM'].count()
    EncounterMax = max(EncounterTotal)
    EncounterMean = np.round(np.mean(EncounterTotal), 2)
    EncounterSTD = np.round(np.std(EncounterTotal), 2)
    print("ICD max: ", EncounterMax)
    print("ICD mean: ", EncounterMean)
    print("ICD Standard deviation: ", EncounterSTD)
    
    return PatientIcdCodes, TotalIcdCode

In [ ]:
PatientIcdCodes, TotalIcdCodes = Icd_Codes(Diagnosis)
print("Total ICD codes: ", len(TotalIcdCodes))
#assert(len(icd_codes) == 942)

In [ ]:
#Medication statistics
def Medications(Prescriptions):
    #Need advice about medications for data science uses. In nursing, each medication is necessary 
    #however the authors mension removing duplicates per encounter. Should we only be keeping one medication
    #type per encounter? This doesnt seem like a good idea, but I am not sure if that is what the author is 
    #implying. Or, should we keep one medication type per day?
    
    # I am not sure where you saw that but I would assume we would keep each medication from each hospital
    # stay but even removing the same medication from different stays is too high compared to the paper
    

    # use drug code (FORMULARY_DRUG_CD) instead of drug name (DRUG) ?
    
    MedStatistics = Prescriptions[['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'DRUG']]
    MedStatistics = MedStatistics[MedStatistics.SUBJECT_ID.isin(Patients)].reset_index(drop = True)
    
    Num_Reports = MedStatistics[['ROW_ID']]
    print("Number of medication reports: ", len(Num_Reports))
    
    Num_Patients = MedStatistics[['SUBJECT_ID']]
    Num_Patients = Num_Patients.drop_duplicates(['SUBJECT_ID'])
    print("Number of patients with meds: ", len(Num_Patients))
    
    # get list of just possible medications
    All_Meds = MedStatistics[['DRUG']]
    All_Meds = All_Meds.drop_duplicates(['DRUG'])
    print("Number of meds: ", len(All_Meds))
    
    # get drugs per patient, drop duplicate drugs for the same patient
    NewMedStatistics = MedStatistics[['SUBJECT_ID', 'HADM_ID', 'DRUG']]
    MedsPerPatient = NewMedStatistics.drop_duplicates()
    
    MedsPerPatient = MedsPerPatient.groupby(["SUBJECT_ID", "HADM_ID"])["DRUG"].apply(list)
    print(MedsPerPatient.head(10))
    
    #print(MedsPerPatient)
    
    return MedsPerPatient, All_Meds

In [ ]:
print("Patient count: ", len(Patients))
MedsPerPatient, All_Meds = Medications(Prescriptions)

# Without SUBJECT_ID filter
# Number of medication reports:  1048575
# Number of patients with meds:  10333
# Number of meds:  2726
# Medication count: 10333

# With SUBJECT_ID filter
# Number of medication reports:  539950
# Number of patients with meds:  2397
# Number of meds:  2213
# Medication count: 2397

print("Medication count: ", len(MedsPerPatient))
# how many medications they ended up with in appendix 1
#assert(len(meds) == 3202)
# Medications
# min per encounter: 0
# max per encounter: 164

In [ ]:
#Lab component statistics. Mostly interested in abnormal labs

def Laboratory(LabEvents):
    #Only include the abnormal flags, and ignore the actual values.
    LabStatistics = LabEvents[['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'FLAG']]
    IcuPatientLab = LabStatistics[LabStatistics.SUBJECT_ID.isin(Patients)].reset_index(drop = True)
    TotalCodes = IcuPatientLab.filter(["ITEMID"]).drop_duplicates()
    
    #Create a list of all labs if needed
    AllLabs = IcuPatientLab.groupby("SUBJECT_ID")["ITEMID"].apply(list)
    
    #Separate the lab events into all found in ICU patients. not sure if needed
    IcuLabList = IcuPatientLab.drop_duplicates(['ITEMID'])
    
    #Gather only the abnormal flagged events
    IcuPatientLab = IcuPatientLab.dropna().reset_index(drop = True)
    IcuPatientLabList = []
    TotalAbnormalCodes = IcuPatientLab.filter(["ITEMID"]).drop_duplicates()
    IcuPatientLab.HADM_ID = IcuPatientLab.HADM_ID.astype('int')
    AbnormalLabs = IcuPatientLab.groupby(["SUBJECT_ID", "HADM_ID"])["ITEMID"].apply(list)

    #Abnormal lab statistics by patient
    LabTotal = IcuPatientLab.drop_duplicates(["SUBJECT_ID", "ITEMID"]).groupby('SUBJECT_ID')['ITEMID'].count()
    LabMax = max(LabTotal)
    LabMean = np.round(np.mean(LabTotal), 2)
    LabSTD = np.round(np.std(LabTotal), 2)
    print("Abnormal Lab max: ", LabMax)
    print("Abnormal Lab mean: ", LabMean)
    print("Abnormal Lab Standard deviation: ", LabSTD)
    
    return AbnormalLabs, TotalAbnormalCodes, TotalCodes

In [ ]:
AbnormalLabs, TotalAbnormalCodes, TotalCodes = Laboratory(LabEvents)
print("Total abnormal lab codes: ", len(TotalAbnormalCodes))
print("Total lab codes: ", len(TotalCodes))
#assert(len(TotalAbnormalCodes == 284))
#assert(len(TotalCodes) == 681)

In [ ]:
#Outcome label. Creates a list of 0's or 1's of length Patients, where 1's indicate the patient passed away.

def Mortality(Admissions, Patients):
    OutcomeFlag = Admissions[['SUBJECT_ID', 'HOSPITAL_EXPIRE_FLAG']]
    Outcome = OutcomeFlag[OutcomeFlag.SUBJECT_ID.isin(Patients)].drop_duplicates('SUBJECT_ID').reset_index(drop = True)
    Deceased = []

    #Create list of deceased patients from ICU only patients
    for i in Outcome.to_numpy():
        if i[1] == 1:
            Deceased.append(i[0])
    
    #convert to numpy and organize
    Deceased = np.array(Deceased)
    Deceased = pd.DataFrame(Deceased, columns = ['Deceased']).to_numpy()
    DeceasedFlag = []
    
    #Create list of 0s and 1s to determine from Patients which patient survived and which did not.
    for i in Patients:
        if i in Deceased:
            DeceasedFlag.append(1)
        else:
            DeceasedFlag.append(0)
            
    return DeceasedFlag


In [ ]:
outcome = Mortality(Admissions, Patients)

In [ ]:
def PrepareData(PatientIcdCodes, MedsPerPatient, AbnormalLabs):
    #Reform into dataframes
    IcdCodes = pd.DataFrame(PatientIcdCodes)
    MedCodes = pd.DataFrame(MedsPerPatient)
    LabCodes = pd.DataFrame(AbnormalLabs)

    #Merge all together keeping both patient id and visits separate
    IcdMeds = pd.merge(IcdCodes, MedCodes, how = "left", on = ["SUBJECT_ID", "HADM_ID"])
    IcdMedsLabs = pd.merge(IcdMeds, LabCodes, how = "left", on = ["SUBJECT_ID", "HADM_ID"])
    IcdMedsLabs = pd.merge(IcdMeds, LabCodes, how = "left", on = ["SUBJECT_ID", "HADM_ID"])
    IcdMedsLabs = IcdMedsLabs.fillna("").apply(list)
    
    # combine codes now to make the list conversion easier
    #IcdMedsLabs['CODES']= IcdMedsLabs.values.tolist()
    IcdMedsLabs = IcdMedsLabs.reset_index()
    display(IcdMedsLabs.head(5))
    ICD_CODES = IcdMedsLabs[["SUBJECT_ID", "HADM_ID", "ICD9_CODE"]]
    MED_CODES = IcdMedsLabs[["SUBJECT_ID", "HADM_ID", "DRUG"]]
    LAB_CODES = IcdMedsLabs[["SUBJECT_ID", "HADM_ID", "ITEMID"]]
    
    IcdList = ICD_CODES.groupby("SUBJECT_ID")["ICD9_CODE"].apply(list)
    IcdList = IcdList.tolist()
    
    MedList = MED_CODES.groupby("SUBJECT_ID")["DRUG"].apply(list)
    MedList = MedList.tolist()
    
    LabList = LAB_CODES.groupby("SUBJECT_ID")["ITEMID"].apply(list)
    LabList = LabList.tolist()
    
    return IcdList, MedList, LabList

In [ ]:
IcdList, MedList, LabList = PrepareData(PatientIcdCodes, MedsPerPatient, AbnormalLabs)
print(IcdList[0][0])
print(MedList[0][0])
print(LabList[0][0])

In [ ]:
#Next, we implement the custom dataset with our sequence variable
class CustomDataset(Dataset):
    
    def __init__(self, IcdList, MedList, LabList, outcome):
        self.Icd = IcdList
        self.Meds = MedList
        self.Labs = LabList
        self.y = outcome
    
    
    def __len__(self):
        return len(outcome)
    
    
    def __getitem__(self, index):
        return self.Icd[index], self.Meds[index], self.Labs[index], self.y[index]

In [ ]:
#run dataset
dataset = CustomDataset(IcdList, MedList, LabList, outcome)
print(dataset[1])

In [ ]:
#This collate function is currently experimental. It may or may not be the correct way to implement 
#a collate function for multiple datasets of information.
#The alternate method would be to use the course HW collate function, feed ICD, meds, and labs in
#Individually and then concatenate them after. Unsure currently.

def custom_collate_fn(data):

    sequences, labels = zip(*data)

    y = torch.tensor(labels, dtype=torch.float)
    
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    #num_meds = [len(visit) for ]
    #num_labs = ...
    
    max_num_visits = max(num_visits)
    max_num_icd = max(num_codes)
    max_num_meds = max(num_meds)
    max_num_labs = max(num_labs)
    
    icd_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    med_x = torch.zeros((num_patients, max_num_visits, max_num_meds), dtype=torch.long)
    lab_x = torch.zeros((num_patients, max_num_visits, max_num_labs), dtype=torch.long)
    
    reverse_icdx = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    reverse_medx = torch.zeros((num_patients, max_num_visits, max_num_meds), dtype=torch.long)
    reverse_labx = torch.zeros((num_patients, max_num_visits, max_num_labs), dtype=torch.long)
    
    icd_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    med_masks = torch.zeros((num_patients, max_num_visits, max_num_meds), dtype=torch.bool)
    lab_masks = torch.zeros((num_patients, max_num_visits, max_num_labs), dtype=torch.bool)
    
    reverse_icd_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    reverse_med_masks = torch.zeros((num_patients, max_num_visits, max_num_meds), dtype=torch.bool)
    reverse_lab_masks = torch.zeros((num_patients, max_num_visits, max_num_labs), dtype=torch.bool)
    
    
    #need to alter this completely
    for i_patient, patient in enumerate(sequences):
        for j_visit, visit in enumerate(patient):
            """
            TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
            """
            tot_p = len(patient)
            tot_v = len(visit)
            x[i_patient, j_visit, :tot_v] = torch.tensor(visit, dtype = torch.long)
            rev_x[i_patient, tot_p-j_visit-1, :tot_v] = torch.tensor(visit, dtype = torch.long)
            masks[i_patient, j_visit, :tot_v].fill_(1)
            rev_masks[i_patient, tot_p-j_visit-1, :tot_v].fill_(1)
            
            
    
    return icd_x, med_x, lab_x, reverse_icdx, reverse_medx, reverse_labx, icd_masks, med_masks, lab_masks, reverse_icd_masks, reverse_med_masks, reverse_lab_masks, y

In [ ]:
#retain customdataset

#Need to rename the other class to avoid errors.

class CustomDataset(Dataset):
    
    def __init__(self, ICD_Codes, outcome):
        self.x = ICD_Codes
        self.y = outcome
    
    def __len__(self):
        
        """
        TODO: Return the number of samples (i.e. patients).
        """
        
        return len(self.x)
    
    def __getitem__(self, index):
        
        """
        TODO: Generates one sample of data.
        
        Note that you DO NOT need to covert them to tensor as we will do this later.
        """
        
        return self.x[index], self.y[index]
        

dataset = CustomDataset(IcdList, outcome)

In [ ]:
#This is used for our ICD only Retain which is described below

def retain_collate_fn(data):

    sequences, labels = zip(*data)

    y = torch.tensor(labels, dtype=torch.float)
    
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    
    x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    rev_x = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.long)
    masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    rev_masks = torch.zeros((num_patients, max_num_visits, max_num_codes), dtype=torch.bool)
    for i_patient, patient in enumerate(sequences):
        for j_visit, visit in enumerate(patient):
            """
            TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
            """
            tot_p = len(patient)
            tot_v = len(visit)
            x[i_patient, j_visit, :tot_v] = torch.tensor(visit, dtype = torch.long)
            rev_x[i_patient, tot_p-j_visit-1, :tot_v] = torch.tensor(visit, dtype = torch.long)
            masks[i_patient, j_visit, :tot_v].fill_(1)
            rev_masks[i_patient, tot_p-j_visit-1, :tot_v].fill_(1)
    
    return x, masks, rev_x, rev_masks, y

In [ ]:
#Split our data into train, validate and test as per the papers values
split = int(len(dataset) * 0.7)
split2 = int(len(dataset) * 0.1)
split3 = int(len(dataset) * 0.2002)
lengths = [split, split2, split3]
print(len(dataset))
print(sum(lengths))
print(lengths)
train_dataset, val_dataset, test_dataset = random_split(dataset, lengths)

print("Length of training dataset: ", len(train_dataset))
print("Length of validation dataset: ", len(val_dataset))
print("Length of test dataset: ", len(test_dataset))

In [ ]:
#Data loader to create train validation and test splits
def load_data(train_dataset, val_dataset, test_dataset, retain_collate_fn):

    batch_size = 32
    train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, collate_fn = retain_collate_fn)
    val_loader = DataLoader(dataset = val_dataset, batch_size = batch_size, collate_fn = retain_collate_fn)
    test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, collate_fn = retain_collate_fn)
    
    return train_loader, val_loader, test_loader

In [ ]:
train_loader, val_loader, test_loader = load_data(train_dataset, val_dataset, test_dataset, retain_collate_fn)

In [ ]:
#Retain model - ICD codes only






In [ ]:
class AlphaAttention(torch.nn.Module):

    def __init__(self, hidden_dim):
        super().__init__()
        self.a_att = nn.Linear(hidden_dim, 1)

    def forward(self, g):
        x = self.a_att(g)
        softmax = torch.nn.Softmax(dim = 1)
        x = softmax(x)
        return x       

In [ ]:
class BetaAttention(torch.nn.Module):

    def __init__(self, hidden_dim):
        super().__init__()
        self.b_att = nn.Linear(hidden_dim, hidden_dim)


    def forward(self, h):
        x = self.b_att(h)
        x = torch.tanh(x)
        return x

In [ ]:
def attention_sum(alpha, beta, rev_v, rev_masks):
    rev_masks = torch.unsqueeze(torch.max(rev_masks, dim = 2)[0], dim = -1)
    c = alpha * beta * (rev_v * rev_masks)
    c = torch.sum(c, dim = 1)
    return c    

In [ ]:
def sum_embeddings_with_mask(x, masks):
    x = x * masks.unsqueeze(-1)
    x = torch.sum(x, dim = -2)
    return x

In [ ]:
class RETAIN(nn.Module):
    
    def __init__(self, num_codes, embedding_dim=128):
        super().__init__()
        # Define the embedding layer using `nn.Embedding`. Set `embDimSize` to 128.
        self.embedding = nn.Embedding(num_codes, embedding_dim)
        # Define the RNN-alpha using `nn.GRU()`; Set `hidden_size` to 128. Set `batch_first` to True.
        self.rnn_a = nn.GRU(embedding_dim, embedding_dim, batch_first=True)
        # Define the RNN-beta using `nn.GRU()`; Set `hidden_size` to 128. Set `batch_first` to True.
        self.rnn_b = nn.GRU(embedding_dim, embedding_dim, batch_first=True)
        # Define the alpha-attention using `AlphaAttention()`;
        self.att_a = AlphaAttention(embedding_dim)
        # Define the beta-attention using `BetaAttention()`;
        self.att_b = BetaAttention(embedding_dim)
        # Define the linear layers using `nn.Linear()`;
        self.fc = nn.Linear(embedding_dim, 1)
        # Define the final activation layer using `nn.Sigmoid().
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x, masks, rev_x, rev_masks):
        # 1. Pass the reversed sequence through the embedding layer;
        rev_x = self.embedding(rev_x)
        # 2. Sum the reversed embeddings for each diagnosis code up for a visit of a patient.
        rev_x = sum_embeddings_with_mask(rev_x, rev_masks)
        # 3. Pass the reversed embegginds through the RNN-alpha and RNN-beta layer separately;
        g, _ = self.rnn_a(rev_x)
        h, _ = self.rnn_b(rev_x)
        # 4. Obtain the alpha and beta attentions using `AlphaAttention()` and `BetaAttention()`;
        alpha = self.att_a(g)
        beta = self.att_b(h)
        # 5. Sum the attention up using `attention_sum()`;
        c = attention_sum(alpha, beta, rev_x, rev_masks)
        # 6. Pass the context vector through the linear and activation layers.
        logits = self.fc(c)
        probs = self.sigmoid(logits)
        return probs.squeeze()

# load the model here
retain = RETAIN(num_codes = len(TotalIcdCodes))

In [ ]:
def eval(model, val_loader):
    model.eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x, masks, rev_x, rev_masks, y in val_loader:
        y_logit = model(x, masks, rev_x, rev_masks)
        y_hat = y_logit > 0.5
        y_score = torch.cat((y_score,  y_logit.detach().to('cpu')), dim=0)
        y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
        y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
    
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    roc_auc = roc_auc_score(y_true, y_score)
    return p, r, f, roc_auc

In [ ]:
def train(model, train_loader, val_loader, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for x, masks, rev_x, rev_masks, y in train_loader:
            optimizer.zero_grad()
            y_hat = model(x, masks, rev_x, rev_masks)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        p, r, f, roc_auc = eval(model, val_loader)
        print('Epoch: {} \t Validation p: {:.2f}, r:{:.2f}, f: {:.2f}, roc_auc: {:.2f}'.format(epoch+1, p, r, f, roc_auc))
    return round(roc_auc, 2)

In [ ]:
# load the model
retain = RETAIN(num_codes = len(TotalIcdCodes))

# load the loss function
criterion = nn.BCELoss()
# load the optimizer
optimizer = torch.optim.Adam(retain.parameters(), lr=1e-3)

n_epochs = 10
train(retain, train_loader, val_loader, n_epochs)



In [ ]:
#Now we feed our data into the model
#If it does not fit into the Autoencoder easily, we should implement it into Retain as our data should 






In [ ]:
#Clout model (The most important model to finish)

#Source: https://github.com/subendhu19/CLOUT

In [ ]:
# Clout outline photos to work with



In [ ]:
# Auto Encoder (AE)

vocabsize_icd = 942
vocabsize_meds = 3202
vocabsize_labs = 284 

class AE(nn.Module):
    def __init__(self, epochs=5, batchsize=50, embsize=100):
        super(AE, self).__init__()
        self.epochs = epochs
        self.batchsize = batchsize
        self.embsize = embsize

        self.emb = nn.Linear(vocabsize_icd + vocabsize_meds + vocabsize_labs, self.embsize)

        self.out = nn.Linear(self.embsize, vocabsize_icd + vocabsize_meds + vocabsize_labs)

        self.reconloss = nn.MSELoss(size_average=True)

    def forward(self, input_icd, input_med, input_lab):

        input_full = torch.cat((input_icd, input_med, input_lab),1)

        hidden_full = F.relu(self.emb(input_full))	 

        output_full = F.relu(self.out(hidden_full))

        return [output_full, hidden_full]

    def get_encodings(self, ICD_data, Lab_data):
        return self.forward(Variable(torch.from_numpy(ICD_data).float()), Variable(torch.from_numpy(Lab_data).float()))[-1]

    def fit(self, ICDs, Meds, Labs):

        optimizer = optim.Adam(self.parameters(), 0.01)

        prev_loss = 1000
        for epoch in range(self.epochs):
            print('Epoch:', epoch)

            perm = np.random.permutation(ICDs.shape[0])
            ICDs = ICDs[perm]
            Meds = Meds[perm]
            Labs = Labs[perm]

            losses = []

            for i in range(0, ICDs.shape[0], self.batchsize):
                ICDbatch, Medbatch, Labbatch = ICDs[i:i+self.batchsize], Meds[i:i+self.batchsize], Labs[i:i+self.batchsize]
                ICDbatchvar, Medbatchvar, Labbatchvar = Variable(torch.from_numpy(ICDbatch).float()), Variable(torch.from_numpy(Medbatch).float()), Variable(torch.from_numpy(Labbatch).float())

                outputs = self.forward(ICDbatchvar, Medbatchvar, Labbatchvar)

                loss = self.reconloss(outputs[0], torch.cat((ICDbatchvar, Medbatchvar, Labbatchvar),1))

                losses.append(loss.data[0])

                optimizer.zero_grad()
               
                loss.backward()
                
                optimizer.step()
                # print 'recon loss:', loss_recon.data[0], 'loss_cr:', loss_cr.data[0]

            print('Epoch loss:', np.mean(losses))

            if abs(np.mean(losses) - prev_loss) < 0.00005:
                break

            prev_loss = np.mean(losses)

model = AE(10,50,175)
ICD_data = pickle.load(open('../full data/CAE/CAEEntries.3digitICD9','r'))
Med_data = pickle.load(open('../full data/CAE/CAEEntries.meds','r'))
Lab_data = pickle.load(open('../full data/CAE/CAEEntries.abnlabs','r'))
model.fit(ICD_data, Med_data, Lab_data)

emb_weights = model._modules['emb'].weight.data.numpy().T
print('Pickled embedding weights. Shape:', np.array(emb_weights).shape)
pickle.dump(emb_weights, open('../full data/CAE/AE_embedding_weights.npy', 'wb'))

# print "Getting embeddings"
# outputs = []

# for i in tqdm(range(0, ICD_data.shape[0], 50)):
# 	ICDbatch, Labbatch = ICD_data[i:i+50], Lab_data[i:i+50]
# 	outputsbatch = model.get_encodings(ICDbatch, Labbatch).data.numpy()
# 	for ob in outputsbatch:
# 		outputs.append(ob)

# pickle.dump(np.array(outputs), open('../full data/CAE/AE_Embeddings', 'wb'))








In [ ]:
# Concatenate Auto Encoder (CAE)

vocabsize_icd = 942
vocabsize_meds = 3202
vocabsize_labs = 284 #all 681 284

class CAE(nn.Module):
    def __init__(self, epochs=5, batchsize=50, embsize=100, lamb=0.01):
        super(CAE, self).__init__()
        self.epochs = epochs
        self.batchsize = batchsize
        self.embsize = embsize
        self.lamb = lamb

        self.emb = nn.Linear(vocabsize_icd + vocabsize_meds + vocabsize_labs, self.embsize)

        self.out = nn.Linear(self.embsize, vocabsize_icd + vocabsize_meds + vocabsize_labs)

        self.reconloss = nn.MSELoss(size_average=True)

    def forward(self, input_icd, input_med, input_lab):

        input_full = torch.cat((input_icd, input_med, input_lab),1)
        input_onlyicd = torch.cat((input_icd, Variable(torch.zeros(input_med.size(0), input_med.size(1)).float()), Variable(torch.zeros(input_lab.size(0), input_lab.size(1)).float())), 1)
        input_onlymed = torch.cat((Variable(torch.zeros(input_icd.size(0), input_icd.size(1)).float()), input_med, Variable(torch.zeros(input_lab.size(0), input_lab.size(1)).float())), 1)
        input_onlylab = torch.cat((Variable(torch.zeros(input_icd.size(0), input_icd.size(1)).float()), Variable(torch.zeros(input_med.size(0), input_med.size(1)).float()), input_lab), 1)

        hidden_full = F.relu(self.emb(input_full))
        hidden_onlyicd = F.relu(self.emb(input_onlyicd))
        hidden_onlymed = F.relu(self.emb(input_onlymed))	
        hidden_onlylab = F.relu(self.emb(input_onlylab))		 

        output_full = F.relu(self.out(hidden_full))
        output_onlyicd = F.relu(self.out(hidden_onlyicd))
        output_onlymed = F.relu(self.out(hidden_onlymed))
        output_onlylab = F.relu(self.out(hidden_onlylab))	

        return [output_full, output_onlyicd, output_onlymed, output_onlylab, hidden_onlyicd, hidden_onlymed, hidden_onlylab, hidden_full]

    def get_encodings(self, ICD_data, Med_data, Lab_data):
        return self.forward(Variable(torch.from_numpy(ICD_data).float()), Variable(torch.from_numpy(Med_data).float()), Variable(torch.from_numpy(Lab_data).float()))[-1]

    def correlation_coef(self, x, y):
        vx = x - torch.mean(x)
        vy = y - torch.mean(y)

        cost = torch.sum(vx * vy) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)))
        return cost

    def joint_cumulant_by_var(self, x, y, z):	
        vx = x - torch.mean(x)
        vy = y - torch.mean(y)
        vz = z - torch.mean(z)

        cost = torch.sum(vx * vy * vz) / (torch.sqrt(torch.sum(vx ** 2)) * torch.sqrt(torch.sum(vy ** 2)) * torch.sqrt(torch.sum(vz ** 2)))
        return cost

        # e_xyz = torch.mean(x * y * z)
        # e_xy = torch.mean(x * y)
        # e_yz = torch.mean(y * z)
        # e_xz = torch.mean(x * z)
        # e_x = torch.mean(x)
        # e_y = torch.mean(y)
        # e_z = torch.mean(z)

        # kappa = e_xyz - (e_xy * e_z) - (e_xz * e_y) - (e_yz * e_x) + (2*e_x*e_y*e_z) 


    def fit(self, ICDs, Meds, Labs):

        optimizer = optim.Adam(self.parameters(), 0.01)

        prev_loss = 1000
        for epoch in range(self.epochs):
            print 'Epoch:', epoch

            perm = np.random.permutation(ICDs.shape[0])
            ICDs = ICDs[perm]
            Meds = Meds[perm]
            Labs = Labs[perm]

            losses = []

            for i in range(0, ICDs.shape[0], self.batchsize):
                ICDbatch, Medbatch, Labbatch = ICDs[i:i+self.batchsize], Meds[i:i+self.batchsize], Labs[i:i+self.batchsize]
                ICDbatchvar, Medbatchvar, Labbatchvar = Variable(torch.from_numpy(ICDbatch).float()), Variable(torch.from_numpy(Medbatch).float()), Variable(torch.from_numpy(Labbatch).float())

                outputs = self.forward(ICDbatchvar, Medbatchvar, Labbatchvar)

                loss_recon = self.reconloss(outputs[0], torch.cat((ICDbatchvar, Medbatchvar, Labbatchvar),1)) + self.reconloss(outputs[1], torch.cat((ICDbatchvar, Medbatchvar, Labbatchvar),1)) \
                        + self.reconloss(outputs[2], torch.cat((ICDbatchvar, Medbatchvar, Labbatchvar),1)) + self.reconloss(outputs[3], torch.cat((ICDbatchvar, Medbatchvar, Labbatchvar),1))

                loss_cr = self.joint_cumulant_by_var(outputs[4], outputs[5], outputs[6])

                loss = loss_recon - (self.lamb*loss_cr)

                losses.append(loss.data[0])

                optimizer.zero_grad()
                    
                loss.backward()
                
                optimizer.step()
                # print 'recon loss:', loss_recon.data[0], 'loss_cr:', loss_cr.data[0]

            print 'Epoch loss:', np.mean(losses)

            if abs(np.mean(losses) - prev_loss) < 0.00005:
                break

            prev_loss = np.mean(losses)


model = CAE(10,50,175,0.01)
ICD_data = pickle.load(open('../full data/CAE/CAEEntries.3digitICD9','r'))
Med_data = pickle.load(open('../full data/CAE/CAEEntries.meds','r'))
Lab_data = pickle.load(open('../full data/CAE/CAEEntries.abnlabs','r'))
model.fit(ICD_data, Med_data, Lab_data)

emb_weights = model._modules['emb'].weight.data.numpy().T
print 'Pickled embedding weights. Shape:', np.array(emb_weights).shape
pickle.dump(emb_weights, open('../full data/CAE/CAE_embedding_weights.npy', 'wb'))

# outputs = model.get_encodings(ICD_data, Med_data, Lab_data)
# print np.array(outputs).shape
# pickle.dump(outputs, open('../full data/CAE/Embeddings', 'wb'))









In [ ]:
#Clout model

class RNN(nn.Module):
    def __init__(self, epochs=5, batchsize=50, vocabsize=5, embsize=100):
        super(RNN, self).__init__()
        self.epochs = 5
        self.batchsize = batchsize
        self.vocabsize = vocabsize
        self.embsize = embsize

        self.emb_icd = nn.Linear(vocabsize_icd, embsize_icd)
        self.emb_meds = nn.Linear(vocabsize_meds, embsize_meds)
        self.emb_labs = nn.Linear(vocabsize_labs, embsize_labs)

        self.rnn = nn.LSTM(input_size=embsize, hidden_size=embsize, num_layers=1)
        self.out = nn.Linear(embsize, 1)
        self.sig = nn.Sigmoid()

    def forward(self, input_icd, input_med, input_lab, input_latent, hidden=None, force=True, steps=0):
        if force or steps == 0: steps = len(input_icd)
        outputs = Variable(torch.zeros(steps, 1, 1))

        input_icd = self.emb_icd(input_icd)
        input_med = self.emb_meds(input_med)
        input_lab = self.emb_labs(input_lab)

        inputs = F.relu(torch.cat((input_icd, input_med, input_lab, input_latent),1))

        inputs = inputs.view(inputs.size()[0],1,inputs.size()[1])
        outputs, hidden = self.rnn(inputs, hidden)
        outputs = self.out(outputs)
        return outputs.squeeze(), hidden

    def predict(self, input_icd, input_med, input_lab, input_latent):
        out, hid = self.forward(input_icd, input_med, input_lab, input_latent, None)
        return self.sig(out[-1]).data[0]

parser = argparse.ArgumentParser(description='Coq tactic prediction')
parser.add_argument('--emb_weights', type=str, default='../full data/CAE/CAE_embedding_weights.npy',
                    help='location of the embedding weight file (default: ../full data/CAE/CAE_embedding_weights.npy)')
args = parser.parse_args()

n_epochs = 1
vocabsize_icd = 942
vocabsize_meds = 3202
vocabsize_labs = 284 #all 681
vocabsize = vocabsize_icd+vocabsize_meds+vocabsize_labs

embsize_icd = 50
embsize_meds = 75
embsize_labs = 50
embsize_latent = 175
embsize = embsize_icd + embsize_labs + embsize_meds + embsize_latent

input_seqs_icd = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.3digitICD9.seqs')))
input_seqs_meds = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.meds.seqs')))
input_seqs_labs = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.abnlabs.seqs')))
# input_seqs_latent = np.array(pickle.load(open('../full data/CAE/Embeddings.seqs')))
latent_weights = pickle.load(open(args.emb_weights))

input_seqs_fullicd = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.seqs')))

icditems = pickle.load(open('../full data/type dictionaries/MIMICIIIPROCESSED.types', 'rb'))
meditems = pickle.load(open('../full data/type dictionaries/MIMICIIIPROCESSED.meds.types', 'rb'))
labitems = pickle.load(open('../full data/type dictionaries/MIMICIIIPROCESSED.abnlabs.types', 'rb'))

model_name = 'AE'
if args.emb_weights == '../full data/CAE/CAE_embedding_weights.npy':
    model_name = 'CAE'

interpretation_file = open("RNN_CLatent_Interpretations_" + model_name + ".txt", 'w')

# overall_risk_factor_file = open("risk_factors_averaged.txt", "w")

labnames = {}
lab_dict_file = open('D_LABITEMS.csv', 'r')
lab_dict_file.readline()
for line in lab_dict_file:
    tokens = line.strip().split(',')
    labnames[tokens[1].replace('"','')] = tokens[2]
lab_dict_file.close()

icdnames = {}
icd_dict_file = open('D_ICD_DIAGNOSES.csv', 'r')
icd_dict_file.readline()
for line in icd_dict_file:
    tokens = line.strip().split(',')
    icdnames[tokens[1].replace('"','')] = tokens[2]
icd_dict_file.close()

icd_scores = {}
med_scores = {}
lab_scores = {}

icd_totals = {}
med_totals = {}
lab_totals = {}

def get_ICD(icd):
    ret_str = ""
    icd_str = icditems.keys()[icditems.values().index(icd)]
    actual_key = icd_str.replace(".", "")[2:]
    if actual_key in icdnames:
        ret_str = icdnames[actual_key]
    else:
        ret_str = icd_str
    return ret_str

def get_med(med):
    ret_str = meditems.keys()[meditems.values().index(med)]
    return ret_str

def get_lab(lab):
    ret_str = labnames[labitems.keys()[labitems.values().index(lab)]]
    return ret_str

print 'Data loaded..'

labels = np.array(pickle.load(open('../full data/MIMICIIIPROCESSED.morts')))

trainratio = 0.7
validratio = 0.1
testratio = 0.2

trainlindex = int(len(input_seqs_icd)*trainratio)
validlindex = int(len(input_seqs_icd)*(trainratio + validratio))

print 'Data prepared..'

def convert_to_one_hot(code_seqs, len):
    new_code_seqs = []
    for code_seq in code_seqs:
        one_hot_vec = np.zeros(len)
        for code in code_seq:
            one_hot_vec[code] = 1
        new_code_seqs.append(one_hot_vec)
    return np.array(new_code_seqs)

def get_avg(seqs, type):
    count = 0
    for seq in seqs:
        count += len(seq)
    val = round(count*1.0/len(seqs))
    if type == 'i':
        return min(4, int(val/5))
    else:
        return min(4, int(val/50))

def get_factors(icd_seq, med_seq, lab_seq, model, actual_score, full_icd):
    potential_test_data = []

    for seq in range(len(icd_seq)):
        for i in range(len(icd_seq[seq])):
            potential_test_data.append(("icd", full_icd[seq][i], seq, icd_seq[:seq]+[icd_seq[seq][:i] + icd_seq[seq][i+1:]]+icd_seq[seq+1:], med_seq, lab_seq))
    for seq in range(len(med_seq)):
        for i in range(len(med_seq[seq])):
            potential_test_data.append(("med", med_seq[seq][i], seq, icd_seq, med_seq[:seq]+[med_seq[seq][:i]+med_seq[seq][i+1:]]+med_seq[seq+1:], lab_seq))
    for seq in range(len(lab_seq)):
        for i in range(len(lab_seq[seq])):
            potential_test_data.append(("lab", lab_seq[seq][i], seq, icd_seq, med_seq, lab_seq[:seq]+[lab_seq[seq][:i] + lab_seq[seq][i+1:]]+lab_seq[seq+1:]))

    risk_scores = []

    for pt in potential_test_data:
        test_input_icd = Variable(torch.from_numpy(convert_to_one_hot(pt[3], vocabsize_icd)).float())
        test_input_med = Variable(torch.from_numpy(convert_to_one_hot(pt[4], vocabsize_meds)).float())
        test_input_lab = Variable(torch.from_numpy(convert_to_one_hot(pt[5], vocabsize_labs)).float())

        latent_inputs_oh = np.concatenate((convert_to_one_hot(pt[3], vocabsize_icd), convert_to_one_hot(pt[4], vocabsize_meds), convert_to_one_hot(pt[5], vocabsize_labs)), 1)
        latent_inputs = np.dot(latent_inputs_oh, latent_weights)
        latent_inputs = Variable(torch.from_numpy(latent_inputs).float())

        factor_score = actual_score - model.predict(test_input_icd, test_input_med, test_input_lab, latent_inputs)
        factor = ""
        if pt[0] == 'icd':
            icd_tag = get_ICD(pt[1])
            factor = "ICD-"+icd_tag
            if icd_tag in icd_scores:
                icd_scores[icd_tag] += factor_score
                icd_totals[icd_tag] += 1
            else:
                icd_scores[icd_tag] = factor_score
                icd_totals[icd_tag] = 1
        elif pt[0] == 'med':
            med_tag = get_med(pt[1])
            factor = "Med-"+med_tag
            if med_tag in med_scores:
                med_scores[med_tag] += factor_score
                med_totals[med_tag] += 1
            else:
                med_scores[med_tag] = factor_score
                med_totals[med_tag] = 1
        else:
            lab_tag = get_lab(pt[1])
            factor = "Lab-"+lab_tag
            if lab_tag in lab_scores:
                lab_scores[lab_tag] += factor_score
                lab_totals[lab_tag] += 1
            else:
                lab_scores[lab_tag] = factor_score
                lab_totals[lab_tag] = 1
        risk_scores.append(("Encounter-"+str(pt[2])+": "+factor, factor_score))

    risk_scores.sort(key=lambda tup: tup[1], reverse=True)

    return risk_scores[:10]

print 'Starting training..'

batchsize = 50

# ICD_wise_tot_tr = np.zeros(5)
# meds_wise_tot_tr = np.zeros(5)
# labs_wise_tot_tr = np.zeros(5)

# for i in range(len(train_input_seqs_icd)):
# 	ICD_wise_tot_tr[get_avg(train_input_seqs_icd[i], 'i')] += 1
# 	meds_wise_tot_tr[get_avg(train_input_seqs_meds[i], 'm')] += 1
# 	labs_wise_tot_tr[get_avg(train_input_seqs_labs[i], 'l')] += 1

# print 'ICD-wise train total', ICD_wise_tot_tr
# print 'Meds-wise train total', meds_wise_tot_tr
# print 'Labs-wise train total', labs_wise_tot_tr

best_aucrocs = []
for run in range(10):
    print 'Run', run

    perm = np.random.permutation(input_seqs_icd.shape[0])
    rinput_seqs_icd = input_seqs_icd#[perm]
    rinput_seqs_meds = input_seqs_meds#[perm]
    rinput_seqs_labs = input_seqs_labs#[perm]
    # rinput_seqs_latent = input_seqs_latent[perm]
    rinput_seqs_fullicd = input_seqs_fullicd#[perm]
    rlabels = labels#[perm]

    train_input_seqs_icd = rinput_seqs_icd[:trainlindex]
    train_input_seqs_meds = rinput_seqs_meds[:trainlindex]
    train_input_seqs_labs = rinput_seqs_labs[:trainlindex]
    # train_input_seqs_latent = rinput_seqs_latent[:trainlindex]
    train_labels = rlabels[:trainlindex]
    train_labels = train_labels.reshape(train_labels.shape[0],1)

    valid_input_seqs_icd = rinput_seqs_icd[trainlindex:validlindex]
    valid_input_seqs_meds = rinput_seqs_meds[trainlindex:validlindex]
    valid_input_seqs_labs = rinput_seqs_labs[trainlindex:validlindex]
    # valid_input_seqs_latent = rinput_seqs_latent[trainlindex:validlindex]
    valid_labels = rlabels[trainlindex:validlindex]

    test_input_seqs_icd = rinput_seqs_icd[validlindex:]
    test_input_seqs_meds = rinput_seqs_meds[validlindex:]
    test_input_seqs_labs = rinput_seqs_labs[validlindex:]
    # test_input_seqs_latent = rinput_seqs_latent[validlindex:]
    test_input_seqs_fullicd = rinput_seqs_fullicd[validlindex:]
    test_labels = rlabels[validlindex:]

    n_iters = train_input_seqs_icd.shape[0]

    model = RNN(n_epochs, 1, vocabsize, embsize)
    criterion = nn.BCEWithLogitsLoss(size_average=False)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    aucrocs = []

    for epoch in range(n_epochs):
        epoch_loss = 0
        print 'Epoch', (epoch+1)

        for i in (range(0, n_iters, batchsize)):
            batch_icd = train_input_seqs_icd[i:i+batchsize]
            batch_meds = train_input_seqs_meds[i:i+batchsize]
            batch_labs = train_input_seqs_labs[i:i+batchsize]
            # batch_latent = train_input_seqs_latent[i:i+batchsize]

            batch_train_labels = train_labels[i:i+batchsize]

            optimizer.zero_grad()
            losses = []

            for i in range(len(batch_icd)):
                icd_onehot = convert_to_one_hot(batch_icd[i], vocabsize_icd)
                med_onehot = convert_to_one_hot(batch_meds[i], vocabsize_meds)
                lab_onehot = convert_to_one_hot(batch_labs[i], vocabsize_labs)

                icd_inputs = Variable(torch.from_numpy(icd_onehot).float())
                med_inputs = Variable(torch.from_numpy(med_onehot).float())
                lab_inputs = Variable(torch.from_numpy(lab_onehot).float())

                latent_inputs_oh = np.concatenate((icd_onehot, med_onehot, lab_onehot), 1)
                latent_inputs = np.dot(latent_inputs_oh, latent_weights)
                latent_inputs = Variable(torch.from_numpy(latent_inputs).float())

                # latent_inputs = Variable(torch.from_numpy(np.array(batch_latent[iter])).float())

                targets = Variable(torch.from_numpy(batch_train_labels[i]).float())

                # Use teacher forcing 50% of the time
                force = random.random() < 0.5
                outputs, hidden = model(icd_inputs, med_inputs, lab_inputs, latent_inputs, None, force)

                #print outputs[-1], targets
                losses.append(criterion(outputs[-1], targets))

            loss = sum(losses)/len(batch_icd)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.data[0]

        #print(epoch, epoch_loss)

        ## Validation phase
        vpredictions = np.zeros(len(valid_input_seqs_icd))
        for i in range(len(valid_input_seqs_icd)):
            test_input_icd = Variable(torch.from_numpy(convert_to_one_hot(valid_input_seqs_icd[i], vocabsize_icd)).float())
            test_input_med = Variable(torch.from_numpy(convert_to_one_hot(valid_input_seqs_meds[i], vocabsize_meds)).float())
            test_input_lab = Variable(torch.from_numpy(convert_to_one_hot(valid_input_seqs_labs[i], vocabsize_labs)).float())

            test_input_latent_oh = np.concatenate((convert_to_one_hot(valid_input_seqs_icd[i], vocabsize_icd), convert_to_one_hot(valid_input_seqs_meds[i], vocabsize_meds), convert_to_one_hot(valid_input_seqs_labs[i], vocabsize_labs)), 1)
            test_input_latent = np.dot(test_input_latent_oh, latent_weights)
            test_input_latent = Variable(torch.from_numpy(test_input_latent).float())

            # test_input_latent = Variable(torch.from_numpy(np.array(valid_input_seqs_latent[i])).float())
            vpredictions[i] = model.predict(test_input_icd, test_input_med, test_input_lab, test_input_latent)

        print "Validation AUC_ROC: ", roc_auc_score(valid_labels, vpredictions)

        ## Testing phase
        predictions = np.zeros(len(test_input_seqs_icd))

        # ICD_wise_corr = np.zeros(5)
        # meds_wise_corr = np.zeros(5)
        # labs_wise_corr = np.zeros(5)
        # ICD_wise_tot = np.zeros(5)
        # meds_wise_tot = np.zeros(5)
        # labs_wise_tot = np.zeros(5)

        for i in range(len(test_input_seqs_icd)):
            test_input_icd = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_icd[i], vocabsize_icd)).float())
            test_input_med = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_meds[i], vocabsize_meds)).float())
            test_input_lab = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_labs[i], vocabsize_labs)).float())

            test_input_latent_oh = np.concatenate((convert_to_one_hot(test_input_seqs_icd[i], vocabsize_icd), convert_to_one_hot(test_input_seqs_meds[i], vocabsize_meds), convert_to_one_hot(test_input_seqs_labs[i], vocabsize_labs)), 1)
            test_input_latent = np.dot(test_input_latent_oh, latent_weights)
            test_input_latent = Variable(torch.from_numpy(test_input_latent).float())

            # test_input_latent = Variable(torch.from_numpy(np.array(test_input_seqs_latent[i])).float())
            predictions[i] = model.predict(test_input_icd, test_input_med, test_input_lab, test_input_latent)
            
            # ICD_wise_corr[get_avg(test_input_seqs_icd[i], 'i')] += int((predictions[i]>0.5)*1 == test_labels[i])
            # ICD_wise_tot[get_avg(test_input_seqs_icd[i], 'i')] += 1

            # meds_wise_corr[get_avg(test_input_seqs_meds[i], 'm')] += int((predictions[i]>0.5)*1 == test_labels[i])
            # meds_wise_tot[get_avg(test_input_seqs_meds[i], 'm')] += 1

            # labs_wise_corr[get_avg(test_input_seqs_labs[i], 'l')] += int((predictions[i]>0.5)*1 == test_labels[i])
            # labs_wise_tot[get_avg(test_input_seqs_labs[i], 'l')] += 1

        print "Test AUC_ROC: ", roc_auc_score(test_labels, predictions)

        aucrocs.append(roc_auc_score(test_labels, predictions))
        #fpr, tpr, _ = roc_curve(test_labels, predictions)
        #pickle.dump({"FPR":fpr, "TPR":tpr}, open('roc_clout_cornn.p', 'wb'))
        #actual_predictions = (predictions>0.5)*1
        #print classification_report(test_labels, actual_predictions)

    best_aucrocs.append(max(aucrocs))

print "Average AUCROC:", np.mean(best_aucrocs), "+/-", np.std(best_aucrocs) 

# print "Final testing and interpretations"
# predictions = np.zeros(len(test_input_seqs_icd))
# for i in (range(len(test_input_seqs_icd))):
# 	test_input_icd = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_icd[i], vocabsize_icd)).float())
# 	test_input_med = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_meds[i], vocabsize_meds)).float())
# 	test_input_lab = Variable(torch.from_numpy(convert_to_one_hot(test_input_seqs_labs[i], vocabsize_labs)).float())
# 	test_input_latent_oh = np.concatenate((convert_to_one_hot(test_input_seqs_icd[i], vocabsize_icd), convert_to_one_hot(test_input_seqs_meds[i], vocabsize_meds), convert_to_one_hot(test_input_seqs_labs[i], vocabsize_labs)), 1)
# 	test_input_latent = np.dot(test_input_latent_oh, latent_weights)
# 	test_input_latent = Variable(torch.from_numpy(test_input_latent).float())

# 	test_score = model.predict(test_input_icd, test_input_med, test_input_lab, test_input_latent)
# 	predictions[i] = test_score
# 	top_risk_factors = get_factors(test_input_seqs_icd[i], test_input_seqs_meds[i], test_input_seqs_labs[i], model, test_score, test_input_seqs_fullicd[i]) 
# 	if (test_score>0.5):
# 		interpretation_file.write("ID: " + str(i) + " True label: "+str(test_labels[i])+"\n")
# 		for rf in top_risk_factors:
# 			interpretation_file.write(str(rf)+"\n")
# 		interpretation_file.write("\n")

interpretation_file.close()

In [ ]:
#Clout training


In [ ]:
#Clout evaluation


In [ ]:
#Summary conclusions


In [ ]:
#Summary Graphs etc...
